In [1]:
import numpy as np
import random
import copy
import threading
import time
import os
import json
from collections import defaultdict
from queue import Queue

tasks = [
    "E", "Q", "G", "N", "F", "K", "M", "H", "I", "L", "S", "O", "B", "C", "P", "D", "J", "R", "A", "T"
]

edges = [
    ("E", "Q"), ("E", "G"), ("E", "N"), ("E", "A"), ("E", "C"), ("E", "R"), ("E", "S"), ("E", "D"), ("E", "B"), ("E", "O"), ("E", "K"), ("E", "P"), ("E", "I"), ("E", "T"), ("E", "J"), ("E", "F"), ("E", "H"),
    ("Q", "R"), ("Q", "J"), ("Q", "D"), ("Q", "T"), ("G", "N"), ("G", "T"), ("N", "R"), ("N", "L"), ("N", "D"), ("N", "K"), ("N", "M"), ("N", "P"), ("N", "F"), ("N", "S"), ("N", "O"), ("N", "H"), ("N", "A"),
    ("N", "C"), ("N", "B"), ("N", "J"), ("N", "I"), ("F", "D"), ("F", "T"), ("F", "C"), ("F", "A"), ("F", "S"), ("F", "J"), ("F", "H"), ("F", "K"), ("F", "O"), ("F", "I"), ("F", "P"), ("F", "M"), ("F", "L"),
    ("K", "T"), ("K", "H"), ("K", "L"), ("K", "A"), ("M", "I"), ("M", "L"), ("M", "P"), ("M", "B"), ("M", "H"), ("M", "D"), ("M", "C"), ("M", "O"), ("M", "T"), ("M", "A"), ("M", "S"), ("M", "R"), ("H", "C"),
    ("H", "B"), ("H", "S"), ("I", "P"), ("I", "A"), ("I", "T"), ("I", "S"), ("I", "J"), ("I", "O"), ("I", "L"), ("I", "C"), ("L", "J"), ("L", "A"), ("L", "D"), ("L", "C"), ("L", "S"), ("S", "R"), ("S", "T"),
    ("S", "A"), ("O", "C"), ("O", "P"), ("O", "A"), ("O", "B"), ("O", "J"), ("O", "R"), ("O", "T"), ("O", "D"), ("B", "A"), ("B", "R"), ("B", "T"), ("B", "P"), ("B", "D"), ("B", "C"), ("B", "J"), ("C", "D"),
    ("P", "T"), ("D", "J"), ("D", "R"), ("J", "R"), ("J", "T"), ("J", "A"), ("R", "A"), ("A", "T")
]

delays = {
    "E": 1, "Q": 5, "G": 5, "N": 0, "F": 0, "K": 2, "M": 2, "H": 5, "I": 3, "L": 1, "S": 5, "O": 4, "B": 4,
    "C": 2, "P": 2, "D": 5, "J": 2, "R": 5, "A": 2, "T": 5
}

def inicialize_graph(edges):
    graph = defaultdict(list)
    predak = defaultdict(list)

    for u, v in edges:
        graph[u].append(v)
        predak[v].append(u)

    return graph, predak

graph, predak = inicialize_graph(edges)

In [2]:
def topological_sort(tasks, edges):
    in_deg = {task: 0 for task in tasks}  # In-degree of each node
    adj_list = defaultdict(list)  # Adjacency list for each node
    
    for u, v in edges:
        adj_list[u].append(v)
        in_deg[v] += 1
    
    next_nodes = [node for node in tasks if in_deg[node] == 0]
    topological_order = []
    
    while next_nodes:
        # Randomly choose a node from the list
        current_node = random.choice(next_nodes)
        next_nodes.remove(current_node)
        topological_order.append(current_node)
        
        for neighbor in adj_list[current_node]:
            in_deg[neighbor] -= 1
            if in_deg[neighbor] == 0:
                next_nodes.append(neighbor)
    
    if len(topological_order) == len(tasks):
        return topological_order
    else:
        raise ValueError("Graph has a cycle, topological sort not possible.")

In [3]:
class Individual:
    def __init__(self, tasks, edges, delay):
        self.schedule = topological_sort(tasks,edges)
        self.fitness = self.calc_fitness(edges, delay)
        self.graph,self.predak = inicialize_graph(edges)

    #ako ne zadovoljava topsort fitness->inf inace izracunaj max(S)
    def calc_fitness(self, edges, delay):
        if not is_valid_schedule(self.schedule, edges):
            return float('-inf')
        #graph, predak = inicialize_graph(edges)
        return -calculate_S(self.schedule, graph, delay, predak)[1]
    
    def invert(self):
        idx1, idx2 = random.sample(range(len(self.schedule)), 2)
        self.schedule[idx1], self.schedule[idx2] = self.schedule[idx2], self.schedule[idx1]
        return idx1, idx2
    
    def __lt__(self, other):
        return self.fitness < other.fitness

In [4]:
def is_valid_schedule(schedule, edges):
    task_to_index = {task: index for index, task in enumerate(schedule)}
    for u, v in edges:
        if task_to_index[u] > task_to_index[v]:
            return False
    return True

In [5]:
def calculate_S(permutation, graph, delay, predak):
    S = {t: 0 for t in permutation}
    for node in permutation:
        max_S = S[node]
        for prec in predak[node]:
            max_S = max(S[node], S[prec] + delay[prec] + 1)

        #ako postoji node sa istim S, uvecamo ga za 1
        while max_S in S.values():
            max_S += 1
        S[node] = max_S

    return S, max(S.values())

In [44]:
def simulatedAnnealing(individual, edges, delays, iters=10000, initial_temp=1.0, alpha=0.99):
    best_individual = copy.deepcopy(individual)
    current_temp = initial_temp

    for _ in range(iters):
        original_schedule = copy.deepcopy(individual.schedule)
        original_fitness = individual.fitness
        
        # Perform a more substantial change (multi-step if necessary)
        idx1, idx2 = individual.invert()
        new_fitness = individual.calc_fitness(edges, delays)
        
        if new_fitness > individual.fitness:
            individual.fitness = new_fitness
        else:
            # Simulated annealing acceptance criteria
            delta_f = new_fitness - individual.fitness
            p = min(1.0, np.exp(delta_f / current_temp))
            q = random.uniform(0, 1)
            if p > q:
                individual.fitness = new_fitness
            else:
                # Revert to original schedule if the change isn't accepted
                individual.schedule = copy.deepcopy(original_schedule)
                individual.fitness = original_fitness
        
        # Update the best individual found
        if individual.fitness > best_individual.fitness:
            best_individual = copy.deepcopy(individual)
        
        # Gradually cool down
        current_temp *= alpha
    
    return best_individual

In [48]:
start_time = time.time()
schedule=Individual(tasks,edges,delays)
best_individual = simulatedAnnealing(schedule, edges, delays)
end_time = time.time()
time_taken = end_time - start_time

print(f'solution: {best_individual.schedule}, cost: {-best_individual.fitness}, time taken: {time_taken}')

solution: ['E', 'G', 'N', 'F', 'Q', 'M', 'K', 'I', 'H', 'O', 'B', 'L', 'S', 'C', 'P', 'D', 'J', 'R', 'A', 'T'], cost: 49, time taken: 0.16606760025024414


### Parallel

In [54]:
def thread_simulatedAnnealing(individual, edges, delays, iters=500, initial_temp=1.0, alpha=0.99):
    best_individual = copy.deepcopy(individual)
    current_temp = initial_temp

    for _ in range(iters):
        original_schedule = copy.deepcopy(individual.schedule)
        original_fitness = individual.fitness
        
        # Perform a more substantial change (multi-step if necessary)
        idx1, idx2 = individual.invert()
        new_fitness = individual.calc_fitness(edges, delays)
        
        if new_fitness > individual.fitness:
            individual.fitness = new_fitness
        else:
            # Simulated annealing acceptance criteria
            delta_f = new_fitness - individual.fitness
            p = min(1.0, np.exp(delta_f / current_temp))
            q = random.uniform(0, 1)
            if p > q:
                individual.fitness = new_fitness
            else:
                # Revert to original schedule if the change isn't accepted
                individual.schedule = copy.deepcopy(original_schedule)
                individual.fitness = original_fitness
        
        # Update the best individual found
        if individual.fitness > best_individual.fitness:
            best_individual = copy.deepcopy(individual)
        
        # Gradually cool down
        current_temp *= alpha
    queue.put(best_individual)
    return best_individual

In [55]:
queue=Queue()
num_threads= 10
threads= []
for i in range(num_threads):
    threads.append(threading.Thread(target=thread_simulatedAnnealing,args=(Individual(tasks,edges,delays),edges,delays)))
    threads[i].start()
best_schedules=set()
for i in range(num_threads):
    threads[i].join()
    best_schedules.add(queue.get())
for best_individual in best_schedules:
    print(f'solution: {best_individual.schedule}, cost: {-best_individual.fitness}, time taken: {time_taken}')

solution: ['E', 'G', 'N', 'Q', 'F', 'M', 'I', 'K', 'H', 'O', 'L', 'S', 'B', 'C', 'P', 'D', 'J', 'R', 'A', 'T'], cost: 49, time taken: 0.16606760025024414
solution: ['E', 'G', 'Q', 'N', 'F', 'M', 'K', 'I', 'H', 'O', 'B', 'L', 'C', 'P', 'S', 'D', 'J', 'R', 'A', 'T'], cost: 49, time taken: 0.16606760025024414
solution: ['E', 'G', 'Q', 'N', 'F', 'M', 'K', 'I', 'H', 'L', 'S', 'O', 'B', 'C', 'P', 'D', 'J', 'R', 'A', 'T'], cost: 50, time taken: 0.16606760025024414
solution: ['E', 'G', 'N', 'Q', 'F', 'M', 'I', 'K', 'H', 'O', 'L', 'B', 'C', 'P', 'D', 'S', 'J', 'R', 'A', 'T'], cost: 49, time taken: 0.16606760025024414
solution: ['E', 'G', 'N', 'Q', 'F', 'M', 'K', 'I', 'H', 'O', 'B', 'L', 'S', 'C', 'D', 'J', 'P', 'R', 'A', 'T'], cost: 49, time taken: 0.16606760025024414
solution: ['E', 'G', 'N', 'F', 'M', 'I', 'K', 'Q', 'H', 'O', 'B', 'L', 'C', 'P', 'D', 'J', 'S', 'R', 'A', 'T'], cost: 49, time taken: 0.16606760025024414
solution: ['E', 'G', 'Q', 'N', 'F', 'M', 'I', 'O', 'K', 'H', 'L', 'S', 'B', 

In [ ]:
def test_alg(path_to_test, path_to_results, alg, big_data=False): 
    files = os.listdir(path_to_test)
    data_files = [f for f in files if f.startswith('test_file_') and f.endswith('.json')]
    
    data_files.sort()
    
    data_to_write = []

    for file_name in data_files:
        file_path = os.path.join(path_to_test, file_name)
        with open(file_path, 'r') as f:
            loaded_data = json.load(f)
            tasks = loaded_data.get('tasks')
            edges = loaded_data.get('edges')
            delays = loaded_data.get('delays')

            order, cost, time_taken = [], float('inf'), float('inf')
            
            # Rough idea of best parameters after performing grid search
            if alg.__name__ == 'run_parallel_ga_sa':
                order, cost, time_taken = alg(tasks, edges, delays)
            else:
                order, cost, time_taken = alg(
                    population_size=40, 
                    num_generations=40, 
                    tournament_size=9,
                    elitism_size=7,
                    mutation_prob=0.5,
                    tasks=tasks,
                    edges=edges,
                    delays=delays
                )
            
            draw_graph(tasks, edges, delays, file_name[:-5] + "_graph", big_data, order)
            print("----------------------------------------------")
            
            data = {
                'test_name': file_name,
                'order_of_tasks': order,
                'finish_time': cost,
                'time_taken': time_taken
            }
            data_to_write.append(data)
    
    save_results(path_to_results, data_to_write)

In [56]:
def draw_graph(tasks, edges, delays, file_name, big_graphs=False, topological_order=None, save_path='graphs/'):
    G = nx.DiGraph()
    
    G.add_nodes_from(tasks)
    G.add_edges_from(edges)
    
    # Dynamic figure size based on the number of nodes
    num_nodes = len(tasks)
    if big_graphs:
        plt.figure(figsize=(12 + num_nodes * 0.2, 12 + num_nodes * 0.2))  # Adjust size dynamically
    else:
        plt.figure(figsize=(8, 8))  # Default size for small graphs
    
    if topological_order:
        # Ensure all nodes are in the topological order
        if len(topological_order) != len(tasks):
            raise ValueError("Topological order does not include all tasks.")
        
        # Use graphviz_layout for hierarchical drawing
        pos = nx.nx_agraph.graphviz_layout(G, prog='dot')
    else:
        pos = nx.spring_layout(G, k=0.5 if big_graphs else 0.8, iterations=50)
    
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    nx.draw(G, pos, with_labels=True, node_color='lightgreen', node_size=1000, font_size=16, font_color='black', arrowstyle='-|>', arrowsize=10)
    
    # Clipping the delays directly below the nodes
    for node, (x, y) in pos.items():
        plt.annotate(f"Delay: {delays[node]}",
                     xy=(x, y), xytext=(0, -20),  # Position text 20 points below the node
                     textcoords='offset points', ha='center', va='center',
                     bbox=dict(facecolor='white', alpha=0.5), fontsize=10, color='blue')

    file_path = os.path.join(save_path, file_name)
    plt.savefig(file_path)
    plt.show()